In [146]:
import os
from api_requests import authenticate_email_api, get_email_content
import pandas as pd
import re
from dateutil.parser import parse

from datetime import date
from dateutil.relativedelta import relativedelta

#lxml required for pandas to parse html tables

In [116]:
os.chdir('/mnt/c/Users/colta/portfolio/codex_vitae_app/config')
    
service = authenticate_email_api()
mynetdiary = get_email_content(service,query="from:no-reply@mynetdiary.net")

print(len(mynetdiary))

22


In [216]:
fitness_table = mynetdiary[1].split("Measurements")[1].replace("\\r\\n","").split("Nutrition")[0]
fitness_df = pd.read_html(fitness_table)[0]

fitness_df['Deep Sleep'] = fitness_df['Awakes,']
fitness_df['Awakes'] = fitness_df['Deep Sleep,']
fitness_df['REM Sleep'] = fitness_df['Sleep,'] - fitness_df['Deep Sleep'] - fitness_df['Light Sleep,']
fitness_df = fitness_df[['Date', 'Weight, lbs', 'BMR, cals','Pulse,', 'Sleep,','Deep Sleep', 'Light Sleep,', 'REM Sleep','Awakes','Daily Steps,', 'Calories Out, cals']]
list(fitness_df.itertuples(index=False,name=None))




[('03/29/21',
  166.4,
  1723.0,
  73,
  4.6,
  2.1,
  1.9,
  0.5999999999999996,
  0.0,
  14336.0,
  2824),
 ('03/30/21', nan, nan, 73, nan, nan, nan, nan, nan, 10449.0, 2736),
 ('03/31/21', nan, nan, 73, 6.7, 1.5, 3.6, 1.6, 0.0, 10775.0, 2735),
 ('04/01/21',
  nan,
  nan,
  70,
  7.5,
  1.4,
  5.0,
  1.0999999999999996,
  0.2,
  10247.0,
  2532),
 ('04/02/21',
  nan,
  nan,
  73,
  9.1,
  3.3,
  4.7,
  1.0999999999999996,
  0.3,
  13286.0,
  2881),
 ('04/03/21', nan, nan, 68, 7.5, 1.3, 6.2, 0.0, 0.1, nan, 226),
 ('04/04/21', nan, nan, 78, 9.9, 0.8, 7.1, 2.0, 0.4, 9924.0, 2211)]

In [209]:
nutrition_table = str(mynetdiary[0]).split("Measurements")[1].replace("\\r\\n","").split("Nutrition")[1].split("Activities")[0]

mon_idx = nutrition_table.find('Monday')
tue_idx = nutrition_table.find('Tuesday')
wed_idx = nutrition_table.find('Wednesday')
thu_idx = nutrition_table.find('Thursday')
fri_idx = nutrition_table.find('Friday')
sat_idx = nutrition_table.find('Saturday')
sun_idx = nutrition_table.find('Sunday')

day_find = [mon_idx,tue_idx,wed_idx,thu_idx,fri_idx,sat_idx,sun_idx]

# Find date strings in HTML table and format as dates.
date_list = [parse(nutrition_table[day:].split("</span")[0]).date() for day in day_find if day != -1]

# Revert to previous year for NYE week.
date_list = [(day - relativedelta(years=1) if date.today().strftime("%m%d") < day.strftime("%m%d") else day) for day in date_list]


nutrition_df = pd.read_html(nutrition_table,parse_dates=True,header=1)

# Retrieve rows containing daily totals.
nutrition_df = nutrition_df[0][nutrition_df[0]['Unnamed: 0'].isnull()].reset_index()
nutrition_df['Date'] = date_list

nutrition_df = nutrition_df[['Date','Calories','Total Fat,\xa0g','Total Carbs,\xa0g', 'Protein,\xa0g',
 'Trans Fat,\xa0g','Saturated Fat,\xa0g','Sodium,\xa0mg', 'Net Carbs,\xa0g', 'Dietary Fiber,\xa0g']]

list(nutrition_df.itertuples(index=False,name=None))

[(datetime.date(2021, 4, 5),
  3067.0,
  '136',
  '301',
  '150',
  '1',
  '28',
  3146.0,
  246.0,
  '53'),
 (datetime.date(2021, 4, 6),
  2331.0,
  '119',
  '140',
  '165',
  nan,
  '19',
  5922.0,
  109.0,
  '32'),
 (datetime.date(2021, 4, 7),
  2088.0,
  '108',
  '144',
  '141',
  nan,
  '18',
  3036.0,
  99.0,
  '44'),
 (datetime.date(2021, 4, 8),
  2713.0,
  '76',
  '346',
  '132',
  nan,
  '41',
  3793.0,
  301.0,
  '42'),
 (datetime.date(2021, 4, 11),
  2150.0,
  '102',
  '185',
  '148',
  nan,
  '20',
  3289.0,
  138.0,
  '45')]

In [ ]:
if __name__ == '__main__':

    os.chdir('/mnt/c/Users/colta/portfolio/codex_vitae_app/config')

    # Search the last two weeks of emails for MyNetDiary nutrition reports.
    date_ = str(date.today()-relativedelta.relativedelta(days=14))
    service = authenticate_email_api_local()
    mynetdiary = get_email_bodies(service,query=f"from:no-reply@mynetdiary.net,after:{date_}")

    #Confirm two weekly reports were returned.
    assert(len(mynetdiary) == 2)